In [1]:
from lohrasb.best_estimator import BaseModel
from optuna.pruners import HyperbandPruner
from optuna.samplers._tpe.sampler import TPESampler
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np
import optuna
from sklearn.pipeline import Pipeline
from feature_engine.imputation import (
    CategoricalImputer,
    MeanMedianImputer
    )
from category_encoders import OrdinalEncoder
from sklearn.metrics import (
    classification_report,
    confusion_matrix,
    f1_score)
from lightgbm import *
from lohrasb.utils.metrics import f1_plus_tn


2023-06-23 23:09:39,726 :: matplotlib :: matplotlib data path: /Users/hjavedani/Documents/Lohrasb/.venv/lib/python3.10/site-packages/matplotlib/mpl-data
2023-06-23 23:09:39,731 :: matplotlib :: CONFIGDIR=/Users/hjavedani/.matplotlib
2023-06-23 23:09:39,734 :: matplotlib :: interactive is False
2023-06-23 23:09:39,735 :: matplotlib :: platform is darwin
2023-06-23 23:09:40,391 :: matplotlib :: CACHEDIR=/Users/hjavedani/.matplotlib
2023-06-23 23:09:40,394 :: matplotlib.font_manager :: Using fontManager instance from /Users/hjavedani/.matplotlib/fontlist-v330.json


#### Example 1 : Use Adult Data Set (a classification problem)
  
https://archive.ics.uci.edu/ml/datasets/Adult

#### Part 1: Use BestModel in sklearn pipeline


In [2]:
urldata= "https://archive.ics.uci.edu/ml/machine-learning-databases/adult/adult.data"
# column names
col_names=["age", "workclass", "fnlwgt" , "education" ,"education-num",
"marital-status","occupation","relationship","race","sex","capital-gain","capital-loss","hours-per-week",
"native-country","label"
]
# read data
data = pd.read_csv(urldata,header=None,names=col_names,sep=',')
data.head()

,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,label
0,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K
1,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K
2,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K
3,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,<=50K
4,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,<=50K


#### Define labels


In [3]:
data.loc[data['label']=='<=50K','label']=0
data.loc[data['label']==' <=50K','label']=0

data.loc[data['label']=='>50K','label']=1
data.loc[data['label']==' >50K','label']=1

data['label']=data['label'].astype(int)

#### Train test split

In [4]:
X = data.loc[:, data.columns != "label"]
y = data.loc[:, data.columns == "label"]
y = y.values.ravel()


X_train, X_test, y_train, y_test =train_test_split(X, y, test_size=0.33, random_state=42)

# for sample_weights
weights = np.ones(len(y_train))


#### Find feature types for later use

In [5]:
int_cols =  X_train.select_dtypes(include=['int']).columns.tolist()
float_cols =  X_train.select_dtypes(include=['float']).columns.tolist()
cat_cols =  X_train.select_dtypes(include=['object']).columns.tolist()


#### Define model and set it argumens 

In [6]:
estimator = LGBMClassifier()
estimator_params = {
        "boosting_type":["gbdt","rf"],
        "boosting_type":["gbdt"],
        "max_depth": [4,5],
        "learning_rate":[0.1, 0.5],
        "random_state":[42],
        "learning_rate":[0.1],
        "importance_type":["split"]

    }

kwargs = {  # params for fit method  
            'fit_random_kwargs' :{
            'sample_weight':None,
            },
            # params for OptunaSearch
            'main_optuna_kwargs' : {
            'estimator':estimator,
            'estimator_params':estimator_params,
            'measure_of_accuracy' :'f1_score(y_true, y_pred,average="weighted")',
            'refit':True,
            },
            'train_test_split_kwargs':{
                'test_size':.3,
                            
            },
            'study_search_kwargs':{
                'storage':None,
                'sampler':TPESampler(),
                'pruner':HyperbandPruner(),
                'study_name':"example of optuna optimizer",
                'direction':"maximize",
                'load_if_exists':False,
            },
            'optimize_kwargs':{
                # optuna optimization params
                'n_trials':20,
                'timeout':600,
                'catch':(),
                'callbacks':None,
                'gc_after_trial':False,
                'show_progress_bar':False,
            }
}




In [7]:
obj = BaseModel().optimize_by_optuna(
            kwargs=kwargs
        )

#### Build sklearn pipeline

In [8]:


pipeline =Pipeline([
            # int missing values imputers
            ('intimputer', MeanMedianImputer(
                imputation_method='median', variables=int_cols)),
            # category missing values imputers
            ('catimputer', CategoricalImputer(variables=cat_cols)),
            #
            ('catencoder', OrdinalEncoder()),
            # classification model
            ('obj', obj),


 ])
 



#### Run Pipeline

In [9]:
pipeline.fit(X_train,y_train)
y_preds = pipeline.predict(X_test)
pred_labels = np.rint(y_preds)




[I 2023-06-23 23:09:41,714] A new study created in memory with name: example of optuna optimizer
[I 2023-06-23 23:09:41,808] Trial 0 finished with value: 0.8625178703940558 and parameters: {'boosting_type': 'gbdt', 'max_depth': 5, 'learning_rate': 0.1, 'random_state': 42, 'importance_type': 'split'}. Best is trial 0 with value: 0.8625178703940558.
[I 2023-06-23 23:09:41,877] Trial 1 finished with value: 0.8620221649136824 and parameters: {'boosting_type': 'gbdt', 'max_depth': 4, 'learning_rate': 0.1, 'random_state': 42, 'importance_type': 'split'}. Best is trial 0 with value: 0.8625178703940558.
[I 2023-06-23 23:09:41,974] Trial 2 finished with value: 0.8625178703940558 and parameters: {'boosting_type': 'gbdt', 'max_depth': 5, 'learning_rate': 0.1, 'random_state': 42, 'importance_type': 'split'}. Best is trial 0 with value: 0.8625178703940558.
[I 2023-06-23 23:09:42,059] Trial 3 finished with value: 0.8625178703940558 and parameters: {'boosting_type': 'gbdt', 'max_depth': 5, 'learning_

2023-06-23 23:09:43,330 :: dev :: If refit is set to True, the optimal model will be refit on the entire dataset, i.e., X_train and y_train!
2023-06-23 23:09:43,330 :: dev :: If refit is set to True, the optimal model will be refit on the entire dataset, i.e., X_train and y_train!


#### Check performance of the pipeline

In [10]:
print('F1 score : ')
print(f1_score(y_test,pred_labels))
print('Classification report : ')
print(classification_report(y_test,pred_labels))
print('Confusion matrix : ')
print(confusion_matrix(y_test,pred_labels))


F1 score : 
0.7022569444444444
Classification report : 
              precision    recall  f1-score   support

           0       0.89      0.95      0.92      8196
           1       0.79      0.63      0.70      2550

    accuracy                           0.87     10746
   macro avg       0.84      0.79      0.81     10746
weighted avg       0.87      0.87      0.87     10746

Confusion matrix : 
[[7756  440]
 [ 932 1618]]


#### Some estimators have predict_proba method as well

In [11]:
y_preds = pipeline.predict_proba(X_test)
print(y_preds)

[[0.99101622 0.00898378]
 [0.57753741 0.42246259]
 [0.34390098 0.65609902]
 ...
 [0.66617167 0.33382833]
 [0.68212666 0.31787334]
 [0.98284642 0.01715358]]


#### Part 2:  Use BestModel as a standalone estimator 

In [12]:
X_train, X_test, y_train, y_test =train_test_split(X, y, test_size=0.33, random_state=42)


#### Transform features to make them ready for model input

In [13]:
transform_pipeline =Pipeline([
            # int missing values imputers
            ('intimputer', MeanMedianImputer(
                imputation_method='median', variables=int_cols)),
            # category missing values imputers
            ('catimputer', CategoricalImputer(variables=cat_cols)),
            #
            ('catencoder', OrdinalEncoder()),
            # classification model

 ])

#### Transform X_train and X_test

In [14]:
X_train=transform_pipeline.fit_transform(X_train,y_train)
X_test=transform_pipeline.transform(X_test)


#### Train model and predict

In [15]:
obj.fit(X_train,y_train)
y_pred = obj.predict(X_test)

[I 2023-06-23 23:09:44,039] A new study created in memory with name: example of optuna optimizer
[I 2023-06-23 23:09:44,331] Trial 0 finished with value: 0.8637983386146815 and parameters: {'boosting_type': 'gbdt', 'max_depth': 5, 'learning_rate': 0.1, 'random_state': 42, 'importance_type': 'split'}. Best is trial 0 with value: 0.8637983386146815.
[I 2023-06-23 23:09:44,432] Trial 1 finished with value: 0.8632261485542435 and parameters: {'boosting_type': 'gbdt', 'max_depth': 4, 'learning_rate': 0.1, 'random_state': 42, 'importance_type': 'split'}. Best is trial 0 with value: 0.8637983386146815.
[I 2023-06-23 23:09:44,519] Trial 2 finished with value: 0.8637983386146815 and parameters: {'boosting_type': 'gbdt', 'max_depth': 5, 'learning_rate': 0.1, 'random_state': 42, 'importance_type': 'split'}. Best is trial 0 with value: 0.8637983386146815.
[I 2023-06-23 23:09:44,590] Trial 3 finished with value: 0.8632261485542435 and parameters: {'boosting_type': 'gbdt', 'max_depth': 4, 'learning_

2023-06-23 23:09:46,720 :: dev :: If refit is set to True, the optimal model will be refit on the entire dataset, i.e., X_train and y_train!
2023-06-23 23:09:46,720 :: dev :: If refit is set to True, the optimal model will be refit on the entire dataset, i.e., X_train and y_train!


#### Check performance of the model

In [16]:
f1_plus_tn
print('F1 score plus TN : ')
print(f1_plus_tn(y_test,pred_labels))
print('F1 score : ')
print(f1_score(y_test,pred_labels))
print('Classification report : ')
print(classification_report(y_test,pred_labels))
print('Confusion matrix : ')
print(confusion_matrix(y_test,pred_labels))

F1 score plus TN : 
7756.702256944444
F1 score : 
0.7022569444444444
Classification report : 
              precision    recall  f1-score   support

           0       0.89      0.95      0.92      8196
           1       0.79      0.63      0.70      2550

    accuracy                           0.87     10746
   macro avg       0.84      0.79      0.81     10746
weighted avg       0.87      0.87      0.87     10746

Confusion matrix : 
[[7756  440]
 [ 932 1618]]


In [17]:
obj.get_best_estimator()

LGBMClassifier(max_depth=5, random_state=42)

In [18]:
obj.best_estimator

LGBMClassifier(max_depth=5, random_state=42)

#### Get fitted randomized search object and its attributes

In [19]:
OptunaObj = obj.get_optimized_object()
print(OptunaObj)

FrozenTrial(number=0, values=[0.8637983386146815], datetime_start=datetime.datetime(2023, 6, 23, 23, 9, 44, 40640), datetime_complete=datetime.datetime(2023, 6, 23, 23, 9, 44, 331546), params={'boosting_type': 'gbdt', 'max_depth': 5, 'learning_rate': 0.1, 'random_state': 42, 'importance_type': 'split'}, distributions={'boosting_type': CategoricalDistribution(choices=('gbdt',)), 'max_depth': IntUniformDistribution(high=5, low=4, step=1), 'learning_rate': UniformDistribution(high=0.1, low=0.1), 'random_state': IntUniformDistribution(high=42, low=42, step=1), 'importance_type': CategoricalDistribution(choices=('split',))}, user_attrs={}, system_attrs={}, intermediate_values={}, trial_id=0, state=TrialState.COMPLETE, value=None)
